# Reflection Agent
As the name suggests, *Reflection* is an AI design pattern where the goal is to iteratively refine the output of an LLM based on a given task. In practice, a reflection workflow typically involves an LLM generation, followed by a reflection, which in itself is another LLM generation conditioned on the output of the first one. For example, given a task to write an article, the first LLM can generate a draft, and the second LLM can generate a critique of the draft article.

A Reflection pattern can be depicted in the following diagram:

<img src="../../imgs/reflection-pattern.png" width="600">

The steps shown in the above diagram are defined as follows:

1. User initiates a task request, e.g. Write a blog post
2. An agent receives the request, routes to the Generator LLM for the generation task. Optionally, the Generator LLM could use a Tool (e.g. knowledge base) to assist in the generation task.
3. The LLM responds with an initial draft.
4. An agent evaluates the initial response, then invokes a Reviewer LLM to reflect on the work from the Generation LLM. The Reviewer LLM critiques the given draft, and responds with suggestions for improvements. Optionally, the Reviewer LLM could use a tool to assist in the reviewing process.
5. An agent receives the response from Reviewer LLM, evaluates the response. It either sends it to the Generator LLM to incorporate the suggestions, or is satisfied with the latest version of the work. This process could iterate multiple times until the Reviewer LLM approves the generated work.
6. The agent returns the final response to the user.


There are 3 agents involved in building this solution: A Generator agent, a Reviewer agent and a Supervisor agent that delegates the tasks between the first two agents. The Generator agent will generate a Press Release (PR) document, and the Reviewer agent will generate a critique of the PR work done by the Generator agent.

In this lab, we'll implement the reflection pattern by using Bedrock Agents. The main task in this example is to create a PR article for an upcoming release of a movie.

Let's get started!

## Amazon Bedrock
Amazon Bedrock Agents offers you the ability to build and configure autonomous agents in your application. An agent helps your end-users complete actions based on organization data and user input. Agents orchestrate interactions between foundation models (FMs), data sources, software applications, and user conversations. In addition, agents automatically call APIs to take actions and invoke knowledge bases to supplement information for these actions. 

First, we are going to create a PR generator agent. This agent is responsible for creating a draft PR for an upcoming movie release.
The agent is also integrated with a Knowledge Base that contains pristine PR examples that have been released in the past by the company. The knowledge base is powered by [Knowledge Bases for Bedrock](https://docs.aws.amazon.com/bedrock/latest/userguide/knowledge-base.html).

Next, we will create a PR reviewer agent. This agent is responsible for performing a detailed review of a given draft PR generated by the PR generator agent.

Finally, we'll create a PR supervisor agent which delegate the responsibility of PR generation and review to the appropriate agents. This agent is also responsible for responding to the user once the final version of the PR is complete.

The following depicts the architecture diagram for this module using Bedrock Agents and Knowledge Bases.

<img src="../../imgs/pr-reflection-agent.png" width="600"> 

## Create a PR Writer agent using Amazon Bedrock Agent
In this section we declare global variables that will act as helpers during entire notebook and you will start to create your first agent.
Here's a diagram that highlights the PR Writer agent which we are going to build:

<img src="../../imgs/pr-reflection-agent-writer.png" width="600"> 



Restore variables from previous notebooks as context for this lab.

In [1]:
%store -r

In [3]:
import boto3
import os
import time

sts_client = boto3.client('sts')
session = boto3.session.Session()

account_id = sts_client.get_caller_identity()["Account"]
region = session.region_name

s3_client = boto3.client('s3', region)
bedrock_client = boto3.client('bedrock-runtime', region)

agent_foundation_model = [
    'us.amazon.nova-pro-v1:0'
]

## Importing helper functions
In this section, we're adding bedrock_agent_helper.py and knowledge_base_helper on Python path, so the files can be recognized and their functionalities can be invoked.

In general, the helper functions handle common tasks including agent creation, Knowledge Bases for Bedrock creation, and accessing data in S3.

In [4]:
import sys

sys.path.insert(0, ".")
sys.path.insert(1, "..")
sys.path.insert(2, "../..")

from utils.bedrock_agent_helper import (
    AgentsForAmazonBedrock
)
from utils.knowledge_base_helper import (
    KnowledgeBasesForAmazonBedrock, upload_directory
)
agents = AgentsForAmazonBedrock()
kb = KnowledgeBasesForAmazonBedrock()

## Creating an Agent

Create the PR agent that will have an `Amazon Bedrock Knowledge Base` integrated where it retrieves the pristine versions of the PRs.

The role of the agent is to create a PR document based on the user's input and potential feedback from the reviewer agent.

We will provide the following instructions to our agent:
```
You are to write a PR news for an upcoming show/movie based on the user request, and an optional recommended changes.

When writing the PR news, you should follow the following guidelines:

1. Headline: Capture Attention & Key Details - Be Concise & Informative: Use active verbs like "announces," "debuts," or "unveils." Include the title, release date/platform, and a hook (e.g., talent or franchise ties).
Example: "A24 Reveals Trailer for Horror Thriller ‘Nightfall’ Starring Florence Pugh, Premiering October 2024."
Highlight Exclusives: If applicable, note premieres at festivals (e.g., "Sundance 2025 Official Selection").

2. Lead Paragraph: The 5 Ws - Summarize Who (studio, talent), What (title, genre), When (release date), Where (platforms/theaters), and Why (unique angle, legacy, or cultural relevance).
Example: "Marvel Studios’ ‘Avengers: Legacy,’ directed by Ryan Coogler, will premiere in theaters globally on May 3, 2025, marking the MCU’s first reboot of the iconic franchise."

3. Key Content Elements - Synopsis: Offer a 1–2 paragraph teaser without spoilers. Emphasize uniqueness (e.g., "a dystopian love story set in 2140’s AI-dominated society").
Talent & Production Credentials: Highlight A-list actors, acclaimed directors, or award-winning crews. Mention prior successes (e.g., "From the Oscar-winning producer of Parasite").
Behind-the-Scenes (BTS) Insights: Share filming locations, technical innovations, or adaptations (e.g., "Shot in Iceland using cutting-edge VR cinematography").

4. Quotes: Add Authenticity - Source quotes from directors, leads, or producers that convey passion or vision.
Example: "‘This film redefines resilience,’ says director Jordan Peele. ‘We’re blending horror with social commentary in ways audiences haven’t seen.’"

5. Multimedia & Links - Embed or link to trailers, stills, and BTS content. Provide access to a password-protected media kit with HD assets.
Include official social handles and hashtags (e.g., #NightfallMovie).

6. Tailor for Target Audiences - Trade Outlets (Deadline, Variety): Focus on box office prospects, production budgets, or industry trends.
Lifestyle/Cultural Press (Vulture, Rolling Stone): Highlight themes, fashion, or soundtracks.
Local Media: Tie in filming locations or local talent.

7. SEO & Discoverability - Use keywords: Title, cast names, genre, and phrases like "upcoming sci-fi movies 2025."
Link to official sites or prior related content (e.g., a franchise’s timeline).

8. Logistics & Contact Info - Clearly state release platforms (e.g., "Streaming exclusively on Netstream starting November 12").
List premiere events or fan screenings.
Provide media contacts: Name, email, phone, and PR firm.

9. Professional Polish - Proofread: Ensure dates, names, and titles are error-free.
Tone: Match the project’s vibe (e.g., playful for comedies, sleek for action).
Style Guide: Adhere to outlet-specific rules (e.g., AP style).

10. Follow-Up & Amplification - Pitch Exclusives: Offer early access to trailers or interviews for top-tier outlets.
Post-Release: Monitor coverage, engage on social media, and update with new assets (e.g., poster drops).

You also have access to a knowledge base that contains pristine, high quality PRs that have been published in the past. You should use the knowledge base in assisting with the writing.

```

We will also connect a knowledge base to retrieve good PR references. Follow the instructions as described below:
```
This knowledge base contains pristine examples of complete PR news. Use this knowledge base to find references high quality PR documents.
```

In [5]:
kb_info = kb.get_kb(lab5_kb_id)
kb_arn = kb_info['knowledgeBase']['knowledgeBaseArn']

Create a configuration for the Knowledge Base. The configuration is to be used in the agent creation process.

In [6]:
kb_config = {
    'kb_id': lab5_kb_id,
    'kb_instruction': """This knowledge base contains prestine examples of complete PR news. Use this knowledge base to find references high quality PR documents."""
}

In [7]:
agent_description = """You are a professional PR news writing agent. You write high quality PR news for upcoming releases for new movie/shows."""
agent_instruction = """You are to write a PR news for an upcoming show/movie based on the user request, and an optional recommended changes.

When writing the PR news, you should follow the following guidelines:

1. Headline: Capture Attention & Key Details - Be Concise & Informative: Use active verbs like "announces," "debuts," or "unveils." Include the title, release date/platform, and a hook (e.g., talent or franchise ties).
Example: "A24 Reveals Trailer for Horror Thriller ‘Nightfall’ Starring Florence Pugh, Premiering October 2024."
Highlight Exclusives: If applicable, note premieres at festivals (e.g., "Sundance 2025 Official Selection").

2. Lead Paragraph: The 5 Ws - Summarize Who (studio, talent), What (title, genre), When (release date), Where (platforms/theaters), and Why (unique angle, legacy, or cultural relevance).
Example: "Marvel Studios’ ‘Avengers: Legacy,’ directed by Ryan Coogler, will premiere in theaters globally on May 3, 2025, marking the MCU’s first reboot of the iconic franchise."

3. Key Content Elements - Synopsis: Offer a 1–2 paragraph teaser without spoilers. Emphasize uniqueness (e.g., "a dystopian love story set in 2140’s AI-dominated society").
Talent & Production Credentials: Highlight A-list actors, acclaimed directors, or award-winning crews. Mention prior successes (e.g., "From the Oscar-winning producer of Parasite").
Behind-the-Scenes (BTS) Insights: Share filming locations, technical innovations, or adaptations (e.g., "Shot in Iceland using cutting-edge VR cinematography").

4. Quotes: Add Authenticity - Source quotes from directors, leads, or producers that convey passion or vision.
Example: "‘This film redefines resilience,’ says director Jordan Peele. ‘We’re blending horror with social commentary in ways audiences haven’t seen.’"

5. Multimedia & Links - Embed or link to trailers, stills, and BTS content. Provide access to a password-protected media kit with HD assets.
Include official social handles and hashtags (e.g., #NightfallMovie).

6. Tailor for Target Audiences - Trade Outlets (Deadline, Variety): Focus on box office prospects, production budgets, or industry trends.
Lifestyle/Cultural Press (Vulture, Rolling Stone): Highlight themes, fashion, or soundtracks.
Local Media: Tie in filming locations or local talent.

7. SEO & Discoverability - Use keywords: Title, cast names, genre, and phrases like "upcoming sci-fi movies 2025."
Link to official sites or prior related content (e.g., a franchise’s timeline).

8. Logistics & Contact Info - Clearly state release platforms (e.g., "Streaming exclusively on Netstream starting November 12").
List premiere events or fan screenings.
Provide media contacts: Name, email, phone, and PR firm.

9. Professional Polish - Proofread: Ensure dates, names, and titles are error-free.
Tone: Match the project’s vibe (e.g., playful for comedies, sleek for action).
Style Guide: Adhere to outlet-specific rules (e.g., AP style).

10. Follow-Up & Amplification - Pitch Exclusives: Offer early access to trailers or interviews for top-tier outlets.
Post-Release: Monitor coverage, engage on social media, and update with new assets (e.g., poster drops).

You also have access to a knowledge base that contains pristine, high quality PRs that have been published in the past. You should use the knowledge base in assisting with the writing.
"""

In [8]:
pr_write_agent_name = "pr_writer_agent"
pr_writer_agent = agents.create_agent(
    pr_write_agent_name,
    agent_description,
    agent_instruction,
    agent_foundation_model,
    kb_arns=[kb_arn],
    code_interpretation=False
)
time.sleep(20) # wait for the agent to be in ready state.
pr_writer_agent

('6KGQ1MJQRX',
 'TSTALIASID',
 'arn:aws:bedrock:us-east-1:602900100639:agent-alias/6KGQ1MJQRX/TSTALIASID',
 'arn:aws:iam::602900100639:role/DEFAULT_AgentExecutionRole')

### Associating knowledge base
Now that we've created the agent, let's associate the previously created knowledge base to it.

In [9]:
agents.associate_kb_with_agent(
    pr_writer_agent[0], # the agent ID
    kb_config['kb_instruction'],
    kb_config['kb_id']
)

## Testing the Agent
At this point, you have successfully created a PR writing agent with given instructions, and a knowledge base that contains references to high quality PR documents. 
Next, we'll test the agent independently to verify that the agent follows the directions and works as intended.
To do so we will use the default alias created: `TSTALIASID` which allows you to invoke a draft version of your agent

In [10]:
%%time
pr_draft_article = agents.invoke(
    """Write a PR news for an upcoming movie given the following information: 
Title: Midnight Vendetta
Tagline: “When the clock strikes twelve, justice wears no mask.”
Genre: Action/Thriller
Release Date: November 22, 2025 (Theatrical & IMAX)

Logline:
A disgraced former MI6 agent infiltrates a glittering Dubai masquerade ball to dismantle a trillion-dollar cyberweapons syndicate—but must confront his deadliest enemy: the traitor who framed him for murder.

Key Production/Cast Details:
Director: David Leitch (Atomic Blonde, Bullet Train)—promises “brutally elegant fight choreography blending Bourne-style close combat with Dubai’s opulent settings.”
Studio: Pika Pictures (Budget: $200M)
Filming Locations: Dubai’s Burj Khalifa, Palm Jumeirah, and a custom-built 360-degree rotating ballroom set for the climactic fight.
Soundtrack: Pulse-pounding score by Ludwig Göransson 
""", 
    pr_writer_agent[0], enable_trace=True,
    trace_level = "all"
)
print("====================")
print(pr_draft_article)

invokeAgent API response object: {'ResponseMetadata': {'RequestId': '0749494e-91cf-4384-a0dc-2e990a2b4920', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Sat, 07 Jun 2025 09:11:26 GMT', 'content-type': 'application/vnd.amazon.eventstream', 'transfer-encoding': 'chunked', 'connection': 'keep-alive', 'x-amzn-requestid': '0749494e-91cf-4384-a0dc-2e990a2b4920', 'x-amz-bedrock-agent-session-id': 'aa48b07d-f45c-4702-bfb4-ea39e91e0390', 'x-amzn-bedrock-agent-content-type': 'application/json'}, 'RetryAttempts': 0}, 'contentType': 'application/json', 'sessionId': 'aa48b07d-f45c-4702-bfb4-ea39e91e0390', 'completion': <botocore.eventstream.EventStream object at 0x14156fc20>}
---
{
  "agentAliasId": "TSTALIASID",
  "agentId": "6KGQ1MJQRX",
  "agentVersion": "DRAFT",
  "callerChain": [
    {
      "agentAliasArn": "arn:aws:bedrock:us-east-1:602900100639:agent-alias/6KGQ1MJQRX/TSTALIASID"
    }
  ],
  "sessionId": "aa48b07d-f45c-4702-bfb4-ea39e91e0390",
  "trace": {
    "orchestrationTrace": {
   

## Create an Alias
We've just completed a test query against the PR Writer agent using the default alias `TSTALIASID`.
The default Alias is a quick way to test out agents before integrating it into your application.
When creating a multi-agent collaboration, it's required to create an Alias explicitly so that it can be used by other agents. This is to ensure the agent is tested, validated, and the functionality is as expected.
Since we've tested and validated our agent, let's now create an alias for it:

In [11]:
pr_writer_alias_id, pr_writer_agent_alias_arn = agents.create_agent_alias(
    pr_writer_agent[0], 'v1'
)

## Saving information
Let's store the environment variables to be used in other labs.

In [12]:
pr_writer_agent_arn = agents.get_agent_arn_by_name(pr_write_agent_name)
pr_writer_agent_id = pr_writer_agent[0]
pr_writer_kb = lab5_knowledge_base_name

%store pr_writer_agent_arn
%store pr_writer_agent_alias_arn
%store pr_writer_alias_id
%store pr_write_agent_name
%store pr_writer_agent_id
%store pr_writer_kb
%store pr_draft_article

Stored 'pr_writer_agent_arn' (str)
Stored 'pr_writer_agent_alias_arn' (str)
Stored 'pr_writer_alias_id' (str)
Stored 'pr_write_agent_name' (str)
Stored 'pr_writer_agent_id' (str)
Stored 'pr_writer_kb' (str)
Stored 'pr_draft_article' (str)


# Next Step
So far, we have created a PR writer agent responsible for creating a draft version of the PR article. In the next notebook, we'll build an agent that reviews and provides feedback for the draft version of the PR to improve the quality of the output.